In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy
import seaborn as sns
import statsmodels.api as sm
import sys
import warnings
from IPython.display import display
from scipy import stats
from sklearn import metrics

import rp2.data
import rp2.regression
from rp2 import hagai_2018

rp2.check_environment()

In [ ]:
np.seterr(all="warn")
warnings.filterwarnings("error", category=RuntimeWarning)

print(f"Python version: {sys.version.split()[0]}")
print(f"Scanpy version: {scanpy.__version__}")
print(f"statsmodels version: {sm.__version__}")

In [ ]:
def display_series(s, indent=0):
    width = s.index.str.len().max() + 2
    for n, v in s.items():
        pad = width - len(n)
        print(f"{' ' * indent}{n}{' ' * pad}{v}")

In [ ]:
condition_columns = ["replicate", "treatment", "time_point"]
index_columns = ["gene"] + condition_columns
time_points = ["0", "2", "4", "6"]

gene_info_df = rp2.load_biomart_gene_symbols_df("mouse")

## Acquisition and preparation of RNA counts

In [ ]:
mouse_umi_adata = hagai_2018.load_umi_counts("mouse")
mouse_umi_adata = mouse_umi_adata[mouse_umi_adata.obs.time_point.isin(time_points)].copy()

print("Full Hagai mouse dataset has:")
print(f"  {mouse_umi_adata.n_obs:,} cells")
print(f"  {mouse_umi_adata.n_vars:,} genes")

assert(mouse_umi_adata.n_obs == 53_086)
assert(mouse_umi_adata.n_vars == 22_048)

del mouse_umi_adata

In [ ]:
mouse_counts_adata = hagai_2018.load_counts("mouse", scaling="median")
mouse_counts_adata = mouse_counts_adata[mouse_counts_adata.obs.time_point.isin(time_points)].copy()

print("Scaled Hagai mouse dataset has:")
print(f"  {mouse_counts_adata.n_vars:,} genes")

assert(mouse_counts_adata.n_obs == 53_086)
assert(mouse_counts_adata.n_vars == 16_798)

In [ ]:
lps_responsive_gene_ids = mouse_counts_adata.var.index[mouse_counts_adata.var.lps_responsive]
print(f"{len(lps_responsive_gene_ids):,} genes are LPS-responsive")

assert(len(lps_responsive_gene_ids) == 2_336)

In [ ]:
additional_gene_symbols = ["Il1b", "Tnf"]
additional_gene_ids = gene_info_df.index[gene_info_df.symbol.isin(additional_gene_symbols)]
analysis_gene_ids = sorted(set(lps_responsive_gene_ids).union(additional_gene_ids))

print(f"{len(analysis_gene_ids):,} genes to be used in analysis")

assert(len(analysis_gene_ids) == 2_338)

In [ ]:
condition_df = mouse_counts_adata.obs[condition_columns].drop_duplicates()

print(f"{len(condition_df)} conditions per gene")
print("Per replicate:")
display_series(condition_df.replicate.value_counts().sort_index(), indent=2)
print(f"{len(condition_df) * len(analysis_gene_ids):,} data points overall")

assert(len(condition_df) == 20)
assert((20 * 2_337) == 46_740)

del condition_df

## Verification of linear mean-variance relationship of mean RNA response

In [ ]:
analysis_count_adata = mouse_counts_adata[:, analysis_gene_ids].copy()
gene_condition_stats_df = hagai_2018.calculate_counts_condition_stats(analysis_count_adata)

assert(len(gene_condition_stats_df) == 46_760)

In [ ]:
def fit_mean_variance_trends(df):
    x = sm.add_constant(df["mean"])
    y = df["variance"]
    model = sm.RLM(y, x, M=sm.robust.norms.HuberT(t=1.345))
    assert(model.M.t == 1.345)

    rlm_results = model.fit()

    results = {
        "intercept": rlm_results.params[0],
        "slope": rlm_results.params[1],
        "intercept_pval": rlm_results.pvalues[0],
        "slope_pval": rlm_results.pvalues[1],
        "r2_unweighted": metrics.r2_score(y, rlm_results.fittedvalues),
        "r2_weighted": metrics.r2_score(y, rlm_results.fittedvalues, sample_weight=rlm_results.weights),
    }
    return pd.Series(results)


treatment_sets = {
    "all": ["unst", "lps", "pic"],
#    "lps": ["unst", "lps"],
#    "pic": ["unst", "pic"],
}

mv_fit_map = {set_name: gene_condition_stats_df[gene_condition_stats_df.treatment.isin(set_list)].groupby("gene").apply(fit_mean_variance_trends)
              for set_name, set_list in treatment_sets.items()}

In [ ]:
all_treatment_mv_fit = mv_fit_map["all"].copy()
all_treatment_mv_fit["accept_intercept"] = all_treatment_mv_fit["intercept_pval"] < 0.05
all_treatment_mv_fit["accept_slope"] = all_treatment_mv_fit["slope_pval"] < 0.05
all_treatment_mv_fit["accept_r2"] = all_treatment_mv_fit["r2_unweighted"] > 0.6
display_series(all_treatment_mv_fit[[c for c in all_treatment_mv_fit.columns if c.startswith("accept_")]].agg(np.count_nonzero))

plt.hist(all_treatment_mv_fit.loc[all_treatment_mv_fit.accept_slope].r2_unweighted, bins=50)
plt.axvline(x=0.6)
plt.xlabel("$R^2$ of slope")
plt.ylabel("Number of genes")
plt.show()

assert(np.count_nonzero(all_treatment_mv_fit.accept_intercept) == 812)
assert(np.count_nonzero(all_treatment_mv_fit.accept_slope) == 2_139)

In [ ]:
all_treatment_good_mv_fit = all_treatment_mv_fit.loc[all_treatment_mv_fit.accept_slope & all_treatment_mv_fit.accept_r2]
all_treatment_good_mv_fit.insert(0, "symbol", gene_info_df.loc[all_treatment_good_mv_fit.index].symbol)

print(f"{len(all_treatment_good_mv_fit):,} mean-variance trends have a good fit (based on slope and unweighted R2)")
print(f"  i.e. {100 * (len(all_treatment_good_mv_fit) / len(analysis_gene_ids)):.1f}%")
print(f"  {np.count_nonzero(all_treatment_good_mv_fit.accept_intercept):,} have a significant intercept")

assert(len(all_treatment_good_mv_fit) == 1_551)
assert(np.count_nonzero(all_treatment_good_mv_fit.accept_intercept) == 564)
assert(np.count_nonzero(all_treatment_good_mv_fit.slope < 0) == 0)

In [ ]:
plt.hist(np.log10(all_treatment_good_mv_fit.slope), bins=100)
plt.xlabel("Mean-variance slope (log$_{10}$)")
plt.ylabel("Number of genes")
plt.show()

#### QUESTION: should we address this outlier?

In [ ]:
display(all_treatment_good_mv_fit.sort_values(by="slope").iloc[-1:])

sns.regplot("mean", "variance", data=gene_condition_stats_df.loc[gene_condition_stats_df.gene == "ENSMUSG00000067149"])
plt.show()

## Fitting of bursting parameters

In [ ]:
txburst_df = rp2.data.load_and_recalculate_txburst_results("mouse", condition_columns, "median")
txburst_df = txburst_df.loc[txburst_df.time_point.isin(time_points)]
txburst_df = txburst_df.loc[txburst_df.gene.isin(all_treatment_good_mv_fit.index)]
txburst_df = txburst_df.copy()

print("For the well-fitted genes:")
print(f"  txburst results are available for {len(txburst_df):,} data points")
print(f"  (across {txburst_df.gene.nunique():,} genes)")

assert(len(txburst_df) == 7_804)
assert(len(txburst_df[condition_columns].drop_duplicates()) == 20)

In [ ]:
txburst_gene_condition_counts = txburst_df.gene.value_counts()
txburst_gene_id_subset = txburst_gene_condition_counts[txburst_gene_condition_counts >= 10]

print(f"{len(txburst_gene_id_subset):,} genes have >= 10 txburst results")

assert(len(txburst_gene_id_subset) == 99)

In [ ]:
data_point_info_df = txburst_df.loc[txburst_df.gene.isin(txburst_gene_id_subset.index)].set_index(index_columns)
data_point_info_df = data_point_info_df.join(gene_condition_stats_df.set_index(index_columns)[["mean"]].add_prefix("rna_")).reset_index()

analysis_mv_fit_df = all_treatment_good_mv_fit.loc[txburst_gene_id_subset.index]

print(f"{len(data_point_info_df):,} data points are available")

assert(data_point_info_df.gene.nunique() == 99)
assert(len(analysis_mv_fit_df) == 99)

In [ ]:
plt.hist(np.log10(analysis_mv_fit_df.slope), bins=20)
plt.xlabel("Mean-variance slope (log$_{10}$)")
plt.ylabel("Number of genes")
plt.show()

In [ ]:
g = sns.jointplot(np.log10(data_point_info_df.bs_point), np.log10(data_point_info_df.bf_point), marginal_kws={"bins": 55})
g.set_axis_labels("Burst size (log$_{10}$)", "Burst frequency (log$_{10}$)")
plt.show()

In [ ]:
plt.scatter(np.log10(data_point_info_df.k_off), np.log10(data_point_info_df.k_on))
plt.plot((-2, 1.5), (-2, 1.5))
plt.xlabel("$k_{off}$ (log$_{10}$)")
plt.ylabel("$k_{on}$ (log$_{10}$)")
plt.show()

#### QUESTION: are the $k_{off}$ values being clipped at 1,000?

I believe the txburst optimisation is limited to a parameter search space of [0, 1000]. However, this doesn't appear to be affecting too many points:

In [ ]:
data_point_info_df.k_off.sort_values().reset_index(drop=True).plot.line()

In [ ]:
data_point_info_df["burstiness"] = data_point_info_df.k_off / data_point_info_df.k_on

plt.hist(np.log10(data_point_info_df["burstiness"]), bins=20)
plt.xlabel("Burstiness (log$_{10}$)")
plt.ylabel("Number of genes")
plt.show()

In [ ]:
def calculate_spearman_r(df, x_var, y_var):
    sp_corr = stats.spearmanr(df[x_var], df[y_var])
    return pd.Series(data={"r": sp_corr.correlation, "r_pval": sp_corr.pvalue})


def create_mean_trend_df(df, var, pval):
    trends_df = df.groupby("gene").apply(calculate_spearman_r, "rna_mean", var)
    trends_df["accept_r"] = trends_df.r < pval
    trends_df["trend"] = "uncertain"
    trends_df.loc[trends_df.accept_r & (trends_df.r > 0), "trend"] = "increasing"
    trends_df.loc[trends_df.accept_r & (trends_df.r < 0), "trend"] = "decreasing"
    return trends_df


for c in ["bf_point", "bs_point", "burstiness"]:
    print(f"Trends for {c} based on Spearman rank correlation:")
    display_series(create_mean_trend_df(data_point_info_df, c, 0.05).trend.value_counts().sort_index(), indent=2)

In [ ]:
def fit_bp_curve(df, y_var):
    x_var = "rna_mean"
    results = rp2.regression.calculate_curve_fit(df, x_var, y_var, loss_function="huber", f_scale=1.0)
    a, b, c = results["a"], results["b"], results["c"]
    if a is np.nan:
        return None
    results["start"], results["end"] = rp2.regression.power_function((df[x_var].min(), df[x_var].max()), a, b, c)
    return pd.Series(data=results)


bp_curve_df_map = {c: data_point_info_df.groupby("gene").apply(fit_bp_curve, c)
                  for c in ["bf_point", "bs_point"]}

In [ ]:
def determine_bp_curve_trends(df):
    series = pd.Series(index=df.index, data="unknown")
    series[df["start"] < df["end"]] = "increasing"
    series[df["start"] > df["end"]] = "decreasing"
    return series


for c, df in bp_curve_df_map.items():
    print(f"Trends for {c} based on curve-fitting:")
    display_series(determine_bp_curve_trends(df).value_counts().sort_index(), indent=2)

## Relationship between burst size and mean-variance gradient of RNA response

TODO

In [ ]:
def predict_bf_and_bs(rna_mean, slope, intercept):
    bs = (slope - 1) + (intercept / rna_mean)
    bf = rna_mean / bs
    return bf, bs


def calculate_bp_prediction_df(df, mv_df):
    slope, intercept = mv_df.loc[df.gene, ["slope", "intercept"]].to_numpy().T
    bf, bs = predict_bf_and_bs(df.rna_mean, slope, intercept)
    return pd.DataFrame(index=df.index, data={"bf": bf, "bs": bs})


predicted_bp_df = calculate_bp_prediction_df(data_point_info_df, analysis_mv_fit_df)

for gene_id, gene_df in data_point_info_df.join(predicted_bp_df.add_suffix("_predicted")).groupby("gene"):
    display(analysis_mv_fit_df.loc[[gene_id]])
    _, axes = plt.subplots(ncols=2, figsize=(10, 4))
    for p, ax in zip(["bs", "bf"], axes):
        ax.scatter(gene_df.rna_mean, gene_df[f"{p}_point"], marker="o", label="inferred")
        ax.scatter(gene_df.rna_mean, gene_df[f"{p}_predicted"], marker="x", label="predicted")
        ax.set_xlabel("RNA mean")
        ax.set_ylabel(p)
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.tight_layout()
    plt.show()

## Modulation of burst size and frequency

TODO

## Relationship between burst size and frequency

TODO